# Exercise 8: Part-of-Speech tagging using Markov Models

We will be doing Part-of-Speech (POS) tagging. This is a common problem in natural language processing that tries to assign grammatical tags to each word in a sentence. 
Often this task is posed as a sequence labeling task that can be modeled as a Markov chain.

Let's consider an example:
We have a sentence $Y_{0:T} = $ `Time flies like an arrow.`  and want to predict the corresponding POS tags that we model as a latent sequence $X_{0:T} =$ `NOUN VERB CONJ DET NOUN`. 
This example already reveals the challenge of this task which requires us to take context information into account, since the word `flies` could also be classified as `NOUN` in another context.


In the following exercise, we will apply the algorithm that we introduced in lecture 15 (slide 36) to this problem. 

### 0. Download and prepare the data

In [1]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from functools import partial

 
# download the treebank corpus from nltk
nltk.download('treebank')
 
# download the universal tagset from nltk
nltk.download('universal_tagset')
 
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

# split data into training and validation set in the ratio 80:20
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\Soenke\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Soenke\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [3]:
# create list of train and test tagged words (ignore sentence boundaries)
train_data = [ tup for sent in train_set for tup in sent ]
test_data = [ tup for sent in test_set for tup in sent ]
print(len(train_data))
print(len(test_data))

80310
20366


In [4]:
# check some of the tagged words.
train_data[:5]
print(train_data[5][1])

ADP


In [5]:
# use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_data}
tag_vocab = {tag: i for i, tag in enumerate(tags)}
 
# check total words in vocabulary
words = {word for word,tag in train_data}
word_vocab = {word: i for i, word in enumerate(words)}

In [6]:
print(tags)
print(tag_vocab)
print(words)
print(word_vocab)

{'NUM', 'PRON', 'ADJ', 'PRT', 'DET', 'ADV', 'NOUN', 'ADP', 'CONJ', '.', 'X', 'VERB'}
{'NUM': 0, 'PRON': 1, 'ADJ': 2, 'PRT': 3, 'DET': 4, 'ADV': 5, 'NOUN': 6, 'ADP': 7, 'CONJ': 8, '.': 9, 'X': 10, 'VERB': 11}
{'cutthroat', '44', 'stock-manipulation', 'Lindner', 'Mostly', 'He', 'Hawaii', 'anyone', 'avenues', 'apiece', 'fine', 'departments', 'scaring', 'Mo.', 'Bangkok', 'diagnosed', 'Phelan', 'attendance', 'posted', 'ushered', 'Heatherington', 'manages', 'cop-killer', 'above-market', 'crash', 'lieutenant', 'Younkers', '275', 'identical', 'Ward', 'beheading', 'introduced', 'guide', 'issuers', 'Bureau', 'high-risk', 'ASSETS', 'Saul', 'daughters', 'negative', 'only', 'payable', 'auspices', 'regime', '700,000', 'high-quality', 'Young', 'punish', '*-142', 'Iran\\/Contra', 'Komatsu', 'nine', 'Shrum', '960', 'hamstrung', 'hefty', 'immune', 'outbid', 'slower', '*T*-70', "'50s", 'match', '18,444', 'Richebourg', 'households', '*T*-3', 'pace', '1.17', '13.15', 'cleaner', 'Vegas', 'Florio', 'pasta', 

## 1. Precomputing probabilities ("Training")  
#### [1] (a) Compute the transition probability
First we need to compute the transition probability, i.e. the probability that one POS tag follows on another: $p(x_t|x_{t-1})$

For this purpose, we can just pre-compute the entire table of conditional probabilities based on the counts from the training set. 

In [7]:
def compute_transition(train_data, tag_vocab):
    p_transition = np.zeros((len(tag_vocab),len(tag_vocab)))
    # TODO

    # Count number of occured transitions
    for i in range(1, len(train_data)):
        curr_tag = tag_vocab[train_data[i][1]]
        prev_tag = tag_vocab[train_data[i-1][1]]
        p_transition[prev_tag, curr_tag]+= 1

    # Normalize counts to obtain valid probabilities
    row_sums = p_transition.sum(axis=1, keepdims=True)
    p_transition /= row_sums

    return p_transition

In [8]:
p_transition = compute_transition(train_data, tag_vocab)
tags_df = pd.DataFrame(p_transition, columns = list(tags), index=list(tags))
display(tags_df)

,NUM,PRON,ADJ,PRT,DET,ADV,NOUN,ADP,CONJ,.,X,VERB
NUM,0.184220,0.001428,0.035345,0.026062,0.003570,0.003570,0.351660,0.037487,0.014281,0.119243,0.202428,0.020707
PRON,0.006834,0.006834,0.070615,0.014123,0.009567,0.036902,0.212756,0.022323,0.005011,0.041913,0.088383,0.484738
ADJ,0.021748,0.000194,0.063301,0.011456,0.005243,0.005243,0.696893,0.080583,0.016893,0.066019,0.020971,0.011456
PRT,0.056751,0.017613,0.082975,0.001174,0.101370,0.009393,0.250489,0.019569,0.002348,0.045010,0.012133,0.401174
DET,0.022855,0.003306,0.206411,0.000287,0.006037,0.012074,0.635906,0.009918,0.000431,0.017393,0.045134,0.040247
ADV,0.029868,0.012025,0.130721,0.014740,0.071373,0.081458,0.032196,0.119472,0.006982,0.139255,0.022886,0.339022
NOUN,0.009144,0.004659,0.012584,0.043935,0.013106,0.016895,0.262344,0.176827,0.042454,0.240094,0.028825,0.149134
ADP,0.063275,0.069603,0.107062,0.001266,0.320931,0.014553,0.323589,0.016958,0.001012,0.038724,0.034548,0.008479
CONJ,0.040615,0.060373,0.113611,0.004391,0.123491,0.057080,0.349067,0.055982,0.000549,0.035126,0.009330,0.150384
.,0.078219,0.068777,0.046137,0.002790,0.172210,0.052575,0.218562,0.092918,0.060086,0.092382,0.025644,0.089700


#### [1] (b) Compute the emission probability
Additionally, we need to compute the emission probability, i.e. the probability that a POS tag is associated with a specific word: $p(y_t|x_t)$.

For this purpose, we can just pre-compute the entire table of conditional probabilities based on the counts from the training set.

In [9]:
def compute_emission(train_data, tag_vocab, word_vocab):
    p_emission = np.zeros((len(word_vocab),len(tag_vocab)))
    # TODO

    for i in range(len(train_data)):
        # obtain row/word:
        row = word_vocab[train_data[i][0]]
        
        # obtain col/tag:
        col = tag_vocab[train_data[i][1]]

        # increment in matrix
        p_emission[row, col] += 1

    # Normalize counts to obtain valid probabilities
    row_sums = p_emission.sum(axis=1, keepdims=True)
    p_emission /= row_sums 
    return p_emission

In [10]:
p_emission = compute_emission(train_data, tag_vocab, word_vocab)
words_df = pd.DataFrame(p_emission, columns = list(tags), index=list(words))
display(words_df)

,NUM,PRON,ADJ,PRT,DET,ADV,NOUN,ADP,CONJ,.,X,VERB
cutthroat,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
stock-manipulation,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lindner,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Mostly,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
senses,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
building-products,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
seven,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
polled,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


#### [1] (c) Compute the unigram probabilities
Finally, we need to compute the unigram probability of observing a given word: $p(y)$.
Analogously for a given POS-tag: $p(x)$

Again, we compute this probability on the counts from the training set.

In [11]:
def compute_unigrams(train_data, word_vocab, tag_vocab):
    p_y = np.zeros(len(word_vocab))
    p_x = np.zeros(len(tag_vocab))
    
    # count words and POS tags in the training data
    for word, tag in train_data:
        if word in word_vocab:
            p_y[word_vocab[word]] += 1
        if tag in tag_vocab:
            p_x[tag_vocab[tag]] += 1
    
    # normalize counts to get probabilities
    p_y = p_y / p_y.sum()
    p_x = p_x / p_x.sum()
    
    return p_y, p_x

In [12]:
p_y, p_x = compute_unigrams(train_data, word_vocab, tag_vocab)
words_df = pd.DataFrame(p_y, index=list(words))
display(words_df)
tag_df = pd.DataFrame(p_x, index=list(tags))
display(tag_df)

,0
cutthroat,0.000025
44,0.000050
stock-manipulation,0.000012
Lindner,0.000012
Mostly,0.000012
...,...
senses,0.000012
building-products,0.000012
seven,0.000087
polled,0.000025


,0
NUM,0.034877
PRON,0.027332
ADJ,0.064127
PRT,0.031814
DET,0.086627
ADV,0.032101
NOUN,0.285967
ADP,0.098394
CONJ,0.022687
.,0.116063


#### Add an unknown token:
Now we computed these probabilities from the training data. Unfortunately evaluating on unseen data can lead to problems if we encounter words that we did not see before. There are many ways to deal with this. One option is to replace unseen words with an `<UNK>` token. We set the emission probability to be uniform, s.t. the algorithm has to rely on the transition probability to figure out the POS-tag.

In [13]:
eps= 1e-12
unk_id = len(word_vocab)
word_vocab["<UNK>"] = unk_id
p_y -= eps
p_y = np.concatenate([p_y,np.array([p_y.shape[0] * eps])], axis=0)
p_emission = np.concatenate([p_emission,np.ones_like(p_emission[:1])/len(p_emission[0])], axis=0)
def turn_unk(w):
    if w not in word_vocab:
        return "<UNK>"
    else:
        return w
test_data = [(turn_unk(w),t) for w,t in test_data]

## [1] 2. Numerical Stability
We have now computed all the probabilities that we need to apply the algorithm from the lecture.
However, if we just apply this algorithm naively, we will run into problems of numerical stability. 
To illustrate this, let's compute the probability of a random sequence of tags $p(x_0,x_1,...,x_N)$:

In [14]:
N = 100 
idx = np.random.choice(list(tag_vocab.values()), size=N+1)
p_seq = np.prod(p_x[idx]) # TODO
print(p_seq)

5.308817802414481e-122


As you can see, although we are multiplying non-zero probabilities, we end up with a probability of zero very quickly as we increase $N$. 

This problem can be avoided by performing computations in log-space. This means that we take the logarithm of all probability values before peforming any computations with them. 
This will turn all products into summations and divisions into subtractions. 
Let's try this trick on the task above and compute the value of $\log p(x_0,x_1,...,x_N)$

In [15]:
log_p_seq = np.sum(np.log(p_x[idx])) # TODO
print(log_p_seq)

-279.2460121708869


Now if we convert this back to $p(x_0,...,x_N)$, by taking the exponential, we will still end up with zero. So what does this help us? 

Here we can make use of the fact that the logarithm is a monotone increasing function, which means in particular that 
$$
\mathrm{argmax}_x p(x) = \mathrm{argmax}_x \log p(x)
$$ 
This is useful in cases like this one where the exact probability is not of interest, but we just want to know the position of the maximum value of a particular function. 

Finally, we need to deal with the situation where we want to take the logarithm of a sum of probabilities. 
In that case, we cannot just "pull the log through". 
Instead we need to use the log-sum-exp trick:
$$
\log \sum_{i=1}^{M} p(x_i) = b + \log \sum_{i=1}^M \exp ((\log p(x_i)) - b)   
$$
with $b = \max_i \log p(x_i)$. 
Please read this short explanation to understand why/how this works: [http://wittawat.com/posts/log-sum_exp_underflow.html](http://wittawat.com/posts/log-sum_exp_underflow.html)

In the following implementation, we will exclusively work with log-probabilities and use the log-sum-exp trick wherever we encounter a sum.
To prepare for this, we now convert all probabilities into log-probabilities and add a small `eps` term to all values to avoid $\log(0)$.

In [16]:
eps= 1e-12
log_p_y = np.log(p_y + eps)
log_p_x = np.log(p_x + eps)
log_p_transition = np.log(p_transition + eps)
log_p_emission = np.log(p_emission + eps)

## 3. The algorithm
Now that we have implemented the emission and transition probabilities, we can use them to compute the predictions $p(x_t|Y)$ for all $t=0,\ldots,n$ by applying the algorithm from the lecture.
Note that we $x$ is a discrete variable (we only have a limited vocabulary $\mathcal{X}$), which allows us to evaluate the integral expressions by carrying out a sum.

#### Please read all subtasks carefully before starting your implementation! 

#### [2] (a) Predict: 
Please compute the prediction step here. Note that you need to apply the log-sum-exp trick here and use numpy operations to efficiently compute the probabilities for all possible $x_t$ at the current timestep $t$. 

$$ p(x_t|Y_{0:t-1}) = \sum_{x_{t-1} \in \mathcal{X}} p(x_t|x_{t-1}) p(x_{t-1}|Y_{0:t-1}) $$

In [17]:
def predict(t, log_p_update, log_p_transition):
    """
    Args:
        t (int): Current timestep.
        log_p_update (np.array): `log p(x_t|y_{0:t})` array of shape [T+1, len(tag_vocab)].
        log_p_transition (np.array): `log p(x_t|x_{t-1})` array of shape [len(tag_vocab), len(tag_vocab)].
    
    Returns: 
        np.array: `log p(x_t|y_{0:t-1})` for timestep t. Array of shape [len(tag_vocab)]
    """
    log_p_predict_unnormalized = np.zeros(len(tag_vocab))

    # To obtain b it must be considered that for every possible tag (x_t) there is a unique b
    # Initialize an array to store b
    b = np.zeros(len(tag_vocab))

    # Search for every possible tag
    for state_t in range(len(tag_vocab)):
        # Initialize b as the smallest value
        b[state_t] = np.finfo(float).min
        # Iterate over possible previous states x_{t-1}
        for state_t_minus_1 in range(len(tag_vocab)):
            # Get the maximum log-summand
            b[state_t] = np.maximum(b[state_t], log_p_update[t-1, state_t_minus_1] + log_p_transition[state_t_minus_1, state_t])

    # Use log-sum-exp trick to avoid numerical instability

    # Initialize log_p_predict with values equal to b
    log_p_predict = b

    # Loop over every possible tag in order to add the log-sums to every possible tag (current state x_t)
    for state_t in range(len(tag_vocab)):
        # To calculate the log-sum, initialize the sum
        total_sum = 0
        
        # Iterate over every possible previous state x_{t-1} to get the sum
        for state_t_minus_1 in range(len(tag_vocab)):
            # Accumulate log probabilities for the current state x_t
            total_sum += np.exp(log_p_update[t-1, state_t_minus_1] + log_p_transition[state_t_minus_1, state_t] - b[state_t])
        
        # Now add the log of the sum
        log_p_predict[state_t] += np.log(total_sum)
    
    return log_p_predict

#### [2] (b) Update: 
Please compute the update step here. Note that you need to perform the operations in log-space here and use numpy operations to efficiently compute the probabilities for all possible $x_t$ at the current timestep $t$. 

$$ p(x_t|Y_{1:t}) = p(y_t|x_t) \frac{p(x_t|Y_{0:t-1})}{p(y_t)} $$

In [18]:
def update(Y, t, log_p_predict, log_p_y, log_p_emission):
    """
    Args:
        Y (np.array): Observed sequence of words in array of shape [T].
        t (int): Current timestep.
        log_p_predict (np.array): `log p(x_t|y_{0:t-1})` array of shape [T+1,len(tag_vocab)].
        log_p_y (np.array): `log p(y_t)` array of shape [len(word_vocab)].
        log_p_emission (np.array): `log p(y_t|x_t)` array of shape [len(word_vocab),len(tag_vocab)].
    
    Returns: 
        np.array: `log p(x_t|y_{0:t})` for timestep t. Array of shape [len(tag_vocab)]
    """
    
    # Initialize an array to store the log probabilities for every possible tag x_t
    log_p_update = np.zeros(len(tag_vocab))
    
    # Calculate the log probabilities for each x_t
    for state_t in range(len(tag_vocab)):
        # Accumulate log probabilities for the current state x_t
        log_p_update[state_t] = log_p_emission[Y[t], state_t] + log_p_predict[t-1, state_t] - log_p_y[Y[t]]

    return log_p_update

#### [2] (c) Smoothing: 
Please compute the smoothing step here. Note that you need to apply the log-sum-exp trick here and use numpy operations to efficiently compute the probabilities for all possible $x_t$ at the current timestep $t$. 

$$ p(x_t|Y) = p(x_t|Y_{0:t}) \sum_{x_{t+1} \in \mathcal{X}} p(x_{t+1}|x_t) \frac{p(x_{t+1}|Y)}{ p(x_{t+1}| Y_{0:t}) } $$

In [19]:
def smoothing(t, log_p_predict, log_p_update, log_p_marginal, log_p_transition): 
    """
    Args:
        t (int): Current timestep.
        log_p_predict (np.array): `log p(x_t|y_{0:t-1})` array of shape [T+1,len(tag_vocab)].
        log_p_update (np.array): `log p(x_t|y_{0:t})` array of shape [T+1,len(tag_vocab)].
        log_p_marginal (np.array): `log p(x_t|Y)` array of shape [T+1,len(tag_vocab)].
        log_p_transition (np.array): `log p(x_t|x_{t-1})` array of shape [len(tag_vocab),len(tag_vocab)].
    
    Returns: 
        np.array: `log p(x_t|Y)` for timestep t. Array of shape [len(tag_vocab)] 
    """

    # To obtain b it must be considered that for every possible tag (x_t) there is a unique b
    # Initialize an array to store b
    b = np.zeros(len(tag_vocab))

    for state_t in range(len(tag_vocab)):
        b[state_t] = np.finfo(float).min
        # Iterate over possible output state x_{t+1}
        for state_t_plus_1 in range(len(tag_vocab)):
            # Get the maximum log-summand
            b[state_t] = np.maximum(b[state_t], log_p_transition[state_t, state_t_plus_1] + log_p_marginal[t+1, state_t_plus_1] - log_p_update[t, state_t_plus_1] )

    # Use log-sum-exp trick to avoid numerical instability

    # Initialize log_p_predict with values equal to b
    log_p_smoothing = b

    # Now add the log-sums to every output (current state x_t)
    for state_t in range(len(tag_vocab)):
        # To calculate the log-sum, initialize the sum
        total_sum = 0
        
        # Iterate over possible future states x_{t+1} to get the sum
        for state_t_plus_1 in range(len(tag_vocab)):
            # Accumulate log probabilities for the current state x_t
            total_sum += np.exp(log_p_transition[state_t, state_t_plus_1] + log_p_marginal[t+1, state_t_plus_1] - log_p_update[t, state_t_plus_1] - b[state_t])
        
        # we need to add the log of the sum as well as log_p_update
        log_p_smoothing[state_t] += log_p_update[t, state_t] + np.log(total_sum)
    
    return log_p_smoothing

#### Plugging it together
The functions above are applied in the following function that executes our algorithm to compute the marginals.
Please do not change anything in this function or the signature of the functions above. 
Take a look at their docstrings to understand what you need to compute.

In [20]:
def compute_marginals(Y_, log_p_transition, log_p_emission, log_p_y, log_p_x, tag_vocab):
    T = len(Y_)
    Y = np.zeros(T+1, dtype=int)
    Y[1:] = Y_ # since we are counting time from 1
    log_p_predict = np.zeros((T+1,len(tag_vocab)))
    log_p_update = np.zeros((T+1,len(tag_vocab)))
    log_p_update[0] = log_p_x  
    for t in range(1,T+1):
        log_p_predict[t] = predict(t, log_p_update, log_p_transition)
        log_p_update[t] = update(Y, t, log_p_predict, log_p_y, log_p_emission)
        
    log_p_marginal = np.zeros((T+1,len(tag_vocab)))
    for t in range(T-1,0,-1):
        log_p_marginal[t] = smoothing(t, log_p_predict, log_p_update, log_p_marginal, log_p_transition) 
    return log_p_marginal[1:]

## 3. Evaluate 
Now we can evaluate the algorithm we implemented on the test data.

Hint: you should get an accuracy > 90% if your implementation is correct.

In [21]:
def to_tokens(X, vocab):
    inv_map = {v: k for k, v in vocab.items()}
    return [inv_map[x] for x in X]

In [22]:
Y_test = np.array([word_vocab[w] for w,t in test_data])
X_test = np.array([tag_vocab[t] for w,t in test_data])
p_X_Y = compute_marginals(Y_test, log_p_transition, log_p_emission, log_p_y, log_p_x, tag_vocab)
X_pred = np.argmax(p_X_Y, axis=1)
acc = np.sum(X_pred == X_test) / X_pred.shape[0] * 100
print(to_tokens(X_test[:10], tag_vocab))
print(to_tokens(X_pred[:10], tag_vocab))
print(to_tokens(Y_test[:10], word_vocab))
print(f"Accuracy: {acc}%")

['DET', 'NOUN', 'VERB', 'X', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP']
['DET', 'NOUN', 'VERB', 'X', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP']
['The', 'company', 'said', '0', 'it', 'is', 'in', 'the', 'process', 'of']
Accuracy: 92.25179220269077%
